In [207]:
from secret_key import togetherapi_key
import os
os.environ['TOGETHER_API_KEY'] = togetherapi_key # set together api key

In [208]:
from langchain_community.llms import Together

# Load Mistral Model
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=128
)
prompt = """### Instruction:
Suggest only one fancy name (no description or explanation) for a restaurant that serves {cuisine} food. Just return the name only.
### Response:"""  # input prompt
response = llm.invoke(prompt) # call the model

print("Suggestion", response.strip())

Suggestion Hibiscus Haven


In [209]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = """### Instruction:
Suggest only one fancy name (no description or explanation) for a restaurant that serves {cuisine} food. Just return the name only.
### Response:"""
)

prompt_template_name.format(cuisine="Italian")

'### Instruction:\nSuggest only one fancy name (no description or explanation) for a restaurant that serves Italian food. Just return the name only.\n### Response:'

In [210]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("American")

'\nSavory Oasis'

In [211]:
# sequential chain (output of one step will be input of another step,
# example: restaurant type = indian so generated restaurant name then give food menu items so output will be food menu )
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="""### Instruction:
Suggest only one fancy name (no description or explanation) for a restaurant that serves {cuisine} food. Just return the name only.
### Response:"""
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='restaurant_name')

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="""### Instruction:
Suggest a list of traditional and popular {cuisine} dishes served at a restaurant named "{restaurant_name}". 
Only return the dish names in bullet-point format like this:

- Dish 1
- Dish 2
- Dish 3
...

No description or extra text. Just the list.

### Response:"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key = 'menu_items')

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)
chain({'cuisine':'Arabic'})

{'cuisine': 'Arabic',
 'restaurant_name': '\nAl-Zahra',
 'menu_items': '\n\n- Biryani\n- Tagine\n- Kebab\n- Falafel\n- Chicken Shawarma\n- Lamb Machboos\n- Tabbouleh\n- Hummus\n- Fattoush\n- Mixed Grill\n- Kunafeh\n- Qatayef\n- Baklava\n- Chocolate Cake\n- Strawberry Cake\n- Apple Cake\n- Banana Cake'}